# General program to check if Full name has account in a platform

1. Query with full name
2. Analyse if full name is in the output webpage and if addtional information like institution is alongside him

Generalities.
1. All plataforms end up with q="Something" -> query(URL_with_q=,QUERY) -> returns html
2. research_gate(html,QUERY) -> QUERY_URLs as a table

## Application to Research Gate

In [424]:
%%writefile drive.cfg
[FILES]
A='B'

Overwriting drive.cfg


In [425]:
import requests
from bs4 import BeautifulSoup
import unidecode
import re
import json
import pandas as pd
import time
import random
import numpy as np
import Levenshtein as lv
import wosplus as wp

File → Publish to the web → Link → Entire Document → Comma separated values 

In [217]:
da=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRfgTblUEXwFD8bm_F4EZX165zX-1hh8hGEeYTINqlB0XWqPW6PPAwXuQUhDoaa7ap9-UEeC4l6EKln/pub?output=csv')

In [38]:
da=da.fillna('').reset_index(drop=True)
da['Full_Name']=(da.NOMBRES+' '+da['PRIMER APELLIDO']+' '+da['SEGUNDO APELLIDO']).str.replace('\s+',' ').str.strip()

In [39]:
da.to_excel('investigadores_udea.xlsx',index=False)

In [3]:
da=da.fillna('').reset_index(drop=True)
da['Full_Name']=(da.NOMBRES+' '+da['PRIMER APELLIDO']+' '+da['SEGUNDO APELLIDO']).str.replace('\s+',' ').str.strip()
da['Query']=da.Full_Name.str.lower().map(unidecode.unidecode)
da['Initials']=da.Query.str.replace(r'^([\w])\w+',r'\1').str.replace(r'\s+([\w])\w+',r' \1')

In [4]:
def extract_keyvalue(key,s):
    import re
    v=''
    v=re.search('"{}":"[\w\.\s\(\)]+"'.format(key),s)
    if v:
        v=v.group(0)
        
    return v

def get_research_gate_data(divs,sleep=1):
    import time
    import json
    rg_data={}        
    pf_url=''
    pf=divs.find("div",{"class":"profile-link"})
    if pf.find('a'):
        if pf.find('a').get('href'):
            pf_url=pf.find('a').get('href')    
        
    if pf_url:
        time.sleep(sleep)
        r=requests.get(pf_url)
    if hasattr(r, 'text'):
        rg_profile=r.text.split("RGCommons")[1]
    if rg_profile:        
        d='{'
        lk=['score','professionalPosition']
        for k in lk:
            sep=','
            if k==lk[-1]:
                sep=''
            kv=extract_keyvalue(k,rg_profile)
            if kv:
                d=d+kv+sep
            else:
                kv='"'+k+'":'+'0'
                d=d+kv+sep            

        d=d+'}'
        try:
            dj=json.loads(d)
        except ValueError:
            dj={}

        try:    
            jj=rg_profile.split('"stats":{"data":')[1].split(',"publicationReadsFormatted"')[0]+'}'
        except IndexError:
            jj='{"citationCount":0,"citationCountFormatted":0,"publicationCount":0,"publicationReads":0,"score":0}'
        
        if jj:
            try: 
                rg_data=json.loads(jj)
            except ValueError:
                rg_data={}
            
        rg_data.update(dj)
        rg_data.update({'RG_Profile':pf_url})
        
    return rg_data

#DEBUG
dap=pd.DataFrame()
for x in ['William Antonio']:#,'Morales Aramburo','Diego Alejandro']:
    dap=dap.append(da[da.Full_Name.str.contains(x)]).reset_index(drop=True)
dap    

#    Implementación

1. Make the query with full name
2. Loop over the pages
3. Select affiliation Universidad de Antioquia
4. Check the name against full name
5. Get the RG profile.

In [ ]:
Full_Name_Search=False # True: Search with all names and all the last names
start=time.time()
max_page=5
#RG has a normalized institution
institution='University of Antioquia' 
# Search for full names
#900:1032;700:800 next: 400-700 
au=da[900:1032].reset_index(drop=True).copy()
if not Full_Name_Search: # Search for one names and one last name
    try:
        name_ii=2
        #0:23 
        au=dfs['nrg_{}'.format(name_ii)].reset_index(drop=True).copy()
    except:
        print('ERROR: define dfs below')
#DEBUG:--------
#au=dap.copy()
#-----------------
#LOG FILE=====
# clean log file
logfile='kkk.txt'
f=open(logfile,'w')
f.write('')
f.close()
#=============
df=pd.DataFrame()
#if True: # Loop over full names
for i in au.index:
    #LOG FILE========
    f=open(logfile,'a')
    f.write('{}\n'.format(i)) # check from the terminal with: tail -f kk.txt
    f.close()
    #================
    fullname=au.loc[i,'Full_Name']
    if not Full_Name_Search:
        fullname=au.loc[i,'NOMBRE COMPLETO']
    query=au.loc[i,'Query']
    initials=au.loc[i,'Initials']
    sleep=random.randint(2,5)
    #----DEBUG ----
    #page=1
    #if True: #loop over page
    #---------------
    for page in range(1,max_page+1):
        if page==1:
            pageurl=''
        else:
            pageurl='&page={}'.format(page)

        url='https://www.researchgate.net/search/authors?q={}{}'.format(query.replace(' ','+'),pageurl)
        time.sleep(sleep)
        r=requests.get(url)
        html=r.text
        search = BeautifulSoup(html,'html.parser')
        mydivs = search.findAll("div", {"class": "account-container"})
        if not mydivs:
            print('Not div class "account-container" for page: {}'.format(page) )
            break
        #----DEBUG ----            
        #n=0
        #if True: # loop over n 
        #--------------
        for n in range(len(mydivs)):
            print('{}, {}'.format(page,n),end='\r')
            name=''
            department=''
            pf_url=''
            n_institution=''
            n_name=''
            rg_data={}
            inst=mydivs[n].find("div",{"class":"institution"})    
            if inst:
                n_institution=inst.contents[0]
            # check_institution(institution,n_institutuion,type=RG) -> bolean (RG has a normalized institution)
            check_institution=True    
            if n_institution!=institution:
                check_institution=False
            # get name:    
            if check_institution:
                na=mydivs[n].find("div",{"class":"name"})
                if hasattr(na, 'text'):
                    name=unidecode.unidecode(na.text).lower().replace('-',' ')
                    n_name=na.text

                dp=mydivs[n].find("div",{"class":"department"})
                if hasattr(dp, 'text'):
                    department=dp.text                
            #check_name(query,mame,type=RG) -> bolean
            if name:
                check_name=True
                for pn in name.split():
                    if pn not in query.split():
                        if len(pn)>2:
                            check_name=False
                        else:
                            if pn[0] not in initials:
                                check_name=False            
            else:
                check_name=False
            if check_name:
                # obtain research gate data
                rg_data=get_research_gate_data(mydivs[n],sleep)
                rg_data.update({'Full_Name':fullname,'Institution':n_institution,'RG_Name':n_name,'RG_Department':department})
                df=df.append(pd.Series(rg_data),ignore_index=True).reset_index(drop=True).fillna('')
                df.to_excel('Research_Gate.xlsx',index=False)
print(time.time()-start)            

Not div class "account-container" for page: 1
Not div class "account-container" for page: 2


name_ii=1

In [561]:
SAVE_OUTPUT=False
#RG=pd.read_excel('Research.xlsx').fillna('')
if not Full_Name_Search: # Search for one names and one last name
    RG=pd.read_excel('Research_Gate_Name_{}_Lastname_1.xlsx'.format(name_ii)).fillna('').reset_index(drop=True)
    
#RG=df.copy()

In [562]:
RG.shape

(91, 15)

Modify output and merge with initial data base

In [563]:
RG=RG.drop(['profileStatsUrl','profileContributionsUrl','profileStatsCitationsUrl','publicationReadsUrl'],axis='columns')

In [564]:
RG.columns

Index(['Full_Name', 'Institution', 'RG_Department', 'RG_Name', 'RG_Profile',
       'citationCount', 'citationCountFormatted', 'professionalPosition',
       'publicationCount', 'publicationReads', 'score'],
      dtype='object')

In [565]:
if not Full_Name_Search: # Search for one names and one last name
    RG['RG_ID']=RG.RG_Profile.str.split('?').str[0].str.split('/').str[-1]

    print(RG.shape)

    RG_FULL=pd.read_excel('Research_Gate_Full.xlsx').fillna('')

    RG_FULL['RG_ID']=RG_FULL.RG_Profile.str.split('?').str[0].str.split('/').str[-1]

    #Check for entries already found

    RG['FN_RG']     =RG.Full_Name     +'_'+RG.RG_ID
    RG_FULL['FN_RG']=RG_FULL.Full_Name+'_'+RG_FULL.RG_ID

    RG_FULL['Query_Type']='Full_Name'

    RG=RG.merge(RG_FULL[['FN_RG','Query_Type']],on='FN_RG',how='left').fillna('')

    RG=RG[RG.Query_Type!='Full_Name'].drop(['RG_ID','FN_RG','Query_Type'],axis='columns')

    print(RG.shape)
    try: 
        da.shape
    except:
        da=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRfgTblUEXwFD8bm_F4EZX165zX-1hh8hGEeYTINqlB0XWqPW6PPAwXuQUhDoaa7ap9-UEeC4l6EKln/pub?output=csv')
        da=da.fillna('').reset_index(drop=True)

    da['Full_Name']=da['NOMBRE COMPLETO']

(91, 12)
(91, 11)


In [566]:
da.shape

(1032, 13)

In [567]:
RGF=RG.merge(da[['Full_Name','DEPARTAMENTO','FACULTAD','GRUPO']],how='left',on='Full_Name')

In [568]:
RGF=RGF[['Full_Name', 'Institution','DEPARTAMENTO','RG_Department', 'RG_Name', 'RG_Profile',
       'citationCount', 'citationCountFormatted', 'professionalPosition',
       'publicationCount', 'publicationReads', 'score','FACULTAD','GRUPO']].reset_index(drop=True)

In [569]:
RGF.shape

(91, 14)

In [587]:
RGE_FULL.shape

(572, 14)

In [586]:
if SAVE_OUTPUT:
#if True:    
    if not Full_Name_Search:
        RGE_FULL.append(RGF).reset_index(drop=True).to_excel(
            'Research_Gate_Full.xlsx',index=False)

In [36]:
if SAVE_OUTPUT:
    if Full_Name_Search:
        RGF.to_excel(
            'Research_Gate_Full.xlsx',index=False)

In [532]:
RGF[RGF.Full_Name.str.lower().map(unidecode.unidecode).str.contains('deny')]

,Full_Name,Institution,DEPARTAMENTO,RG_Department,RG_Name,RG_Profile,citationCount,citationCountFormatted,professionalPosition,publicationCount,publicationReads,score,FACULTAD,GRUPO
0,Deny Juliana Zapata Cardona,University of Antioquia,Escuela de Medicina Veterinaria,Escuela de Medicina Veterinaria,Juliana Zapata,https://www.researchgate.net/profile/Juliana_Zapata3?_sg=P85YbgkbwdbcmgnSQqZH6RA9hHlsCRGGKPE0UFyMwY8lNxMx2lmagS3fzCFawjC4FrLLPggJzUfJEfA,83.0,83,0,1.0,14.0,3.65,Facultad de Ciencias Agrarias,Grupo de Investigación en Patobiología - QUIRÓN


###  Quality control
* Same Fullname
* Same Department

In [588]:
RGFF=pd.read_excel('Research_Gate_Full.xlsx').fillna('').reset_index(drop=True)

In [589]:
RGFF.shape

(663, 14)

In [590]:
RGFF[RGFF.DEPARTAMENTO.str.lower().str.strip().map(unidecode.unidecode)==
     RGFF.RG_Department.str.lower().str.strip().map(unidecode.unidecode)].shape

(197, 14)

In [591]:
RGFF['lv_Full_Name']=RGFF.Full_Name.str.lower().str.strip().map(unidecode.unidecode).combine(\
  RGFF.RG_Name.str.lower().str.strip().map(unidecode.unidecode).str.replace('-',' ') , func=lv.ratio)
RGFF['lv_Department']=RGFF.DEPARTAMENTO.str.lower().str.strip().map(unidecode.unidecode).combine(\
  RGFF.RG_Department.str.lower().str.strip().map(unidecode.unidecode) , func=lv.ratio)

In [592]:
RGFF.shape

(663, 16)

In [593]:
RGFF[np.logical_and(RGFF.lv_Full_Name>0.9,RGFF.lv_Full_Name<0.99)][['Full_Name','RG_Name','DEPARTAMENTO','RG_Department','lv_Full_Name','lv_Department']].reset_index(drop=True)

,Full_Name,RG_Name,DEPARTAMENTO,RG_Department,lv_Full_Name,lv_Department
0,Patricia De Jesus Morales Vega,Patricia de Jesus Morales,Instituto de Física,Instituto de Física,0.909091,1
1,Jesus Francisco Vargas Bonilla,J. Francisco Vargas-Bonilla,Departamento de Ingeniería Electrónica,,0.912281,0
2,Gustavo Antonio Peñuela Mesa,Gustavo Antonio Peñuela,Departamento de Ingeniería Sanitaria y Ambiental,Escuela Ambiental,0.901961,0.393939
3,Julio Cesar Bueno Sanchez,Julio C Bueno-Sanchez,Departamento de Fisiología,Departamento de Fisiología y Bioquímica,0.913043,0.8
4,Walter Dario Cardona Maya,Walter D Cardona Maya,Departamento de Microbiología y Parasitología,Departamento de Microbiología y Parasitología,0.913043,1
5,Maria Del Pilar Jimenez Alzate,María del Pilar Jimenez-A,Departamento de Microbiología y Parasitología,,0.909091,0


In [ ]:
RGFF[RGFF.lv_Full_Name>0.9]

In [595]:
RG_Final,nn=wp.df_split(RGFF,'lv_Full_Name',Operator=">",condition=0.9)

In [596]:
RG_Final.shape,nn.shape

((90, 16), (573, 16))

In [597]:
nn_y,nn_n=wp.df_split(nn,'lv_Department',Operator='>',condition=0.91)

In [598]:
nn_y.shape

(163, 16)

In [599]:
nn_y

,Full_Name,Institution,DEPARTAMENTO,RG_Department,RG_Name,RG_Profile,citationCount,citationCountFormatted,professionalPosition,publicationCount,publicationReads,score,FACULTAD,GRUPO,lv_Full_Name,lv_Department
0,Johann Hasler Perez,University of Antioquia,Departamento de Música,Departamento de Música,Johann Hasler,https://www.researchgate.net/profile/Johann_Hasler?_sg=cdDJqmBDbcLFyze2Q2wAQ-oU7W5CM9iWcBspbHK_OhQiYagHEoR5pXQr35WyOLeY_-dLst-iDSFApio,0,0,0,7,95,3.23,Facultad de Artes,"Artes y Modelos de Pensamiento, Religión, Cultura y Sociedad",0.8125,1
1,Hugo Enrique Espinosa Velasquez,University of Antioquia,Departamento de Música,Departamento de Música,Hugo Espinosa,https://www.researchgate.net/profile/Hugo_Espinosa3?_sg=mmMbDl43MdR49Ybz1xS-qtg17SB0W5iSco_9us9M0kVxhotw7DR1E-U1n8bsJRqZEN0V_P_4mkAoXkg,0,0,0,0,0,0.00,Facultad de Artes,Sin Grupo Asociado,0.590909,1
2,Lavinia Sabina Sorge Radovani,University of Antioquia,Departamento de Teatro,Departamento de Teatro,Lavinia Sorge,https://www.researchgate.net/profile/Lavinia_Sorge?_sg=XuUt_2tWy_7uyiGS_kEu2xW6zMr3re1v9v0RNhhVOodg0YFR1e8drxSbvGutMBHbarTf38wI8xR5H0Y,0,0,0,2,51,1.16,Facultad de Artes,Ártes escénicas y del espectáculo,0.619048,1
3,Lina Maria Villegas Hincapie,University of Antioquia,Departamento de Teatro,Departamento de Teatro,Lina María Villegas,https://www.researchgate.net/profile/Lina_Villegas2?_sg=EEAouWGUCD2vejFm8f_tNC0fRchaazWhezMRBzMyLGNcfypC6DC1OHSOBd_yqx7eXQQYZuqJG1Arwz0,0,0,0,2,904,0.00,Facultad de Artes,"Músicas regionales, Ártes escénicas y del espectáculo",0.808511,1
4,Luis Felipe Viana Cisneros,University of Antioquia,Departamento de Teatro,Departamento de Teatro,Luis Viana,https://www.researchgate.net/profile/Luis_Viana5?_sg=5iLEpR2XnfJ5blgUfAqIkr_5Pt81_09H5sJdBo-2AzkEpK2Fz3nOWRXKzRtk5jRGe_ebVbMYVdr5_8c,0,0,0,2,1,0.00,Facultad de Artes,Sin Grupo Asociado,0.555556,1
5,Martha Eufemia Olivera Angel,University of Antioquia,Escuela de Medicina Veterinaria,Escuela de Medicina Veterinaria,Martha Olivera-Angel,https://www.researchgate.net/profile/Martha_Olivera-Angel?_sg=Z9_qicOBy3aPotAPC07E80_qvLum5uR3pYCWBd6yk5BrWHOHBnhsMhdiA5BUvBsQdmuoJ5B9jVGB0VA,899,899,PROFESOR TITULAR,244,19564,32.98,Facultad de Ciencias Agrarias,Biogénesis,0.833333,1
6,Claudia Patricia Ceballos Fonseca,University of Antioquia,Escuela de Medicina Veterinaria,Escuela de Medicina Veterinaria,Claudia P. Ceballos,https://www.researchgate.net/profile/Claudia_Ceballos?_sg=VEpGG1EmA6l0XMAsTg7ftlbv-u5JMnu9hfA1B9u5gzbCEBzFerCBwHvjvks63CldCFh4Ji4l-9n5JCg,185,185,Professor,20,2186,16.03,Facultad de Ciencias Agrarias,"GaMMA: Genética, Mejoramiento y Modelación Animal, CENTAURO",0.692308,1
7,Marta Isabel Wolff Echeverri,University of Antioquia,Instituto de Biología,Instituto de Biología,Marta Wolff,https://www.researchgate.net/profile/Marta_Wolff2?_sg=miYtbcx5oahr7kBAbueQnMjWANHYnwPhmtRwlF3K1fvTp-9e_l2jjjtEIW1jGyzFSTPs5cirAsNAO_k,1140,"1,140",0,105,10247,30.42,Facultad de Ciencias Exactas y Naturales,Grupo de Entomologia Universidad de Antioquia,0.564103,1
8,Fernando Alveiro Alzate Guarin,University of Antioquia,Instituto de Biología,Instituto de Biología,Fernando Alzate,https://www.researchgate.net/profile/Fernando_Alzate?_sg=R4Ufm3XZu0MDt3hnC6iOdHP65QUGuvvSOiv5eYryv1QruGOqVpARy2RVbOdzEks1D07aDweLjqihii4,301,301,Professor (Associate),50,3143,23.73,Facultad de Ciencias Exactas y Naturales,Grupo de Estudios Botánicos,0.666667,1
9,Luz Fernanda Jimenez Segura,University of Antioquia,Instituto de Biología,Instituto de Biología,Luz Jimenez,https://www.researchgate.net/profile/Luz_Jimenez4?_sg=cRujsMa9o0WUwyK5QBeK6_Kkc3dfkH6fw7Uhy8TjEAjBM9WFKNzslkWbxas_I9Kxc72g917MWV7Y-20,375,375,0,49,12074,21.24,Facultad de Ciencias Exactas y Naturales,Grupo de Ictiología,0.578947,1


In [430]:
RG_Final=RG.append(nn_y).reset_index(drop=True)

In [434]:
RG_Final['match']='YES'

In [446]:
RG_Final.shape

(219, 21)

In [455]:
dat=da.merge(RG_Final[['Full_Name','match','RG_Name']],left_on='NOMBRE COMPLETO',right_on='Full_Name',how='left').fillna('')

In [456]:
repeat_search=dat[dat.match==''].drop('match',axis='columns').reset_index(drop=True)

## Search again for low quality results

In [479]:
nrg=repeat_search
nrg.shape

(857, 14)

In [480]:
nrg=nrg[nrg.RG_Name==''].reset_index(drop=True)
# Recover 'Full_Name' columns

In [481]:
# joint second short name with third long name, eg. 'Morelia Del Socorro' to 'Morelia Del-Socorro'
nrg['NOMBRES']=nrg.NOMBRES.str.replace('(\w+\s+)(\w{1,3})\s+(\w+)',r'\1\2-\3')

In [482]:
nrg['Name_number']=nrg.NOMBRES.str.split().map(len)

In [483]:
max_names=nrg['Name_number'].max()

In [484]:
max_names

3

In [485]:
for i in range(max_names):
    print(i)
    nrg['NAME_{}'.format(i+1)]=nrg.NOMBRES.str.split().str[i].fillna('').str.replace('-',' ')

0
1
2


In [486]:
dfs=pd.Series()
for i in range(max_names):
    dfs['nrg_{}'.format(i+1)]=nrg[nrg.Name_number==i+1].reset_index(drop=True)

In [488]:
# redefine nrg_1 to take into account all first names
dfs.nrg_1=nrg.copy()

In [490]:
for i in range(1,max_names+1):
    print(i)
    dfs['nrg_{}'.format(i)]['Full_Name']=(dfs['nrg_{}'.format(i)]['NAME_{}'.format(i)]+' '+dfs[
                                              'nrg_{}'.format(i)]['PRIMER APELLIDO']).str.strip()
    dfs['nrg_{}'.format(i)]['Query']    =dfs['nrg_{}'.format(i)].Full_Name.str.lower().map(unidecode.unidecode)
    dfs['nrg_{}'.format(i)]['Initials'] =dfs['nrg_{}'.format(i)].Query.str.replace(
                r'^([\w])\w+',r'\1').str.replace(r'\s+([\w])\w+',r' \1')

    dfs['nrg_{}'.format(i)]=dfs['nrg_{}'.format(i)][['NOMBRE COMPLETO','Full_Name','Query','Initials',
                                                     'FACULTAD','DEPARTAMENTO','GRUPO']]

1
2
3


In [493]:
dfs['nrg_{}'.format(1)].shape,dfs['nrg_{}'.format(2)].shape

((857, 7), (679, 7))

In [492]:
dfs['nrg_{}'.format(3)]

,NOMBRE COMPLETO,Full_Name,Query,Initials,FACULTAD,DEPARTAMENTO,GRUPO
0,Maria Beatriz Elena Bernal Rivera,Elena Bernal,elena bernal,e b,Facultad de Artes,Departamento de Artes Visuales,Teoría e Historia del Arte en Colombia
1,Luz Estela Del Sanchez Herrera,Del Sanchez,del sanchez,d s,Facultad de Ciencias Exactas y Naturales,Departamento de Matemáticas,Varios Grupos
2,Juan Manuel Martin Uribe Cano,Martin Uribe,martin uribe,m u,Facultad de Ciencias Sociales y Humanas,Departamento de Psicoanálisis,"Psicoanálisis, Sujeto y Sociedad"
3,Diego Jose Luis Botia Valderrama,Luis Botia,luis botia,l b,Facultad de Ingeniería,Departamento de Ingeniería de Sistemas,"Ingeniería y Software , Grupo de Investigación en Telecomunicaciones Aplicadas - GITA, Ingeniería y Tecnologías de las Organizaciones y de la Sociedad (ITOS), Biotecnología Básica y Aplicada"
4,Ramos Juan Nicolas Mc Lean,Nicolas Mc Lean,nicolas mc lean,n m l,Facultad de Medicina,Departamento de Cirugía,Grupo de investigación en cirugía plástica Universidad de Antioquia
5,Luz Ariadna Anamaria Salinas Rendon,Anamaria Salinas,anamaria salinas,a s,Facultad de Medicina,Departamento de Cirugía,Sin Grupo Asociado
6,Maria Paloma Piedad Perez Sastre,Piedad Perez,piedad perez,p p,Facultad de Medicina,Departamento de Educación Médica,Sin Grupo Asociado
7,De Mesa Ysabel Polanco Lopez,Ysabel Polanco,ysabel polanco,y p,Facultad de Medicina,Departamento de Medicina Preventiva y Salud Pública,Respuesta Social en Salud
8,Aguila Manuel Eusebio Molina Del,Eusebio Molina,eusebio molina,e m,Facultad de Medicina,Departamento de Morfología,Sin Grupo Asociado
9,Luis Javier Ignacio Castro Naranjo,Ignacio Castro,ignacio castro,i c,Facultad de Medicina,Departamento de Obstetricia y Ginecología,GIE-SHAE (Grupo Interdisciplinario de Estudio del Síndrome Hipertensivo Asociado al Embarazo)


## Generate output with Quality control

In [616]:
RGFF['Quality']=RGFF[['lv_Department','lv_Full_Name']].max(axis=1).round(3)

In [633]:
RGFF['RG_Profile']=RGFF.RG_Profile.str.split('?').str[0]

In [624]:
RGFF=RGFF[['Quality','Full_Name','RG_Name','Institution', 'DEPARTAMENTO', 'RG_Department', 
        'score','professionalPosition','citationCount', 'citationCountFormatted',
        'publicationCount', 'publicationReads', 
       'FACULTAD', 'GRUPO', 'RG_Profile']]

In [634]:
RGFF[RGFF.score==RGFF.score.max()]

,Quality,Full_Name,RG_Name,Institution,DEPARTAMENTO,RG_Department,score,professionalPosition,citationCount,citationCountFormatted,publicationCount,publicationReads,FACULTAD,GRUPO,RG_Profile
442,0.836,Francisco Javier Lopera Restrepo,Francisco Javier Lopera,University of Antioquia,Departamento de Medicina Interna,Grupo de Neurociencias de Antioquia,45.73,0,5525,"5,525",322,36184,Facultad de Medicina,Grupo de Neurociencias de Antioquia,https://www.researchgate.net/profile/Francisco_Lopera


In [635]:
Q,NQ=wp.df_split(RGFF,on='Quality',Operator='>',condition=0.91)

In [637]:
Q=Q.sort_values('Quality',ascending=False).reset_index(drop=True)

In [ ]:
extra_cols=['PRIMER APELLIDO','SEGUNDO APELLIDO','NOMBRES']
NQ=NQ.merge( da[['Full_Name']+extra_cols],on='Full_Name',how='left'
        ).sort_values(extra_cols).reset_index(drop=False).drop(extra_cols+['index'],axis='columns')

In [669]:
QF=Q.append(NQ.sort_values('Full_Name')).reset_index(drop=True)

In [674]:
QF.to_excel('Research_Gate_Full_Quality.xlsx')

In [671]:
QF.sort_values('score',ascending=False).reset_index(drop=True)[:30]

,Quality,Full_Name,RG_Name,Institution,DEPARTAMENTO,RG_Department,score,professionalPosition,citationCount,citationCountFormatted,publicationCount,publicationReads,FACULTAD,GRUPO,RG_Profile
0,0.836,Francisco Javier Lopera Restrepo,Francisco Javier Lopera,University of Antioquia,Departamento de Medicina Interna,Grupo de Neurociencias de Antioquia,45.73,0,5525,"5,525",322,36184,Facultad de Medicina,Grupo de Neurociencias de Antioquia,https://www.researchgate.net/profile/Francisco_Lopera
1,1.000,Ivan Dario Velez Bernal,Iván D Vélez,University of Antioquia,Instituto de Investigaciones Médicas,Instituto de Investigaciones Médicas,38.61,Director PECET,5508,"5,508",324,8404,Facultad de Medicina,Programa de Estudio y Control de Enfermedades Tropicales,https://www.researchgate.net/profile/Ivan_Velez
2,1.000,Alvaro Enrique Sanabria Quiroga,Alvaro Sanabria,University of Antioquia,Departamento de Cirugía,Departamento de Cirugía,38.25,Associate Professor,1712,"1,712",156,7162,Facultad de Medicina,Trauma y Cirugía,https://www.researchgate.net/profile/Alvaro_Sanabria
3,0.826,Rio Marlene Jimenez Del,Marlene Jimenez Del Rio,University of Antioquia,Instituto de Investigaciones Médicas,Grupo de Neurociencias de Antioquia,36.28,Research Director,1468,"1,468",105,4914,Facultad de Medicina,Grupo de Neurociencias de Antioquia,https://www.researchgate.net/profile/Marlene_Rio
4,1.000,Ivan Dario Florez Gomez,Ivan D Florez,University of Antioquia,Departamento de Pediatría y Puericultura,Departamento de Pediatría y Puericultura,36.21,0,91,91,68,9554,Facultad de Medicina,Grupo de Investigación Clínica en Enfermedades del Niño y del Adolescente - Pediaciencias,https://www.researchgate.net/profile/Ivan_Florez
5,1.000,Mauricio Rojas Lopez,Mauricio Rojas López,University of Antioquia,Instituto de Investigaciones Médicas,Facultad de Medicina,35.59,Professor (Associate),1944,"1,944",122,8713,Facultad de Medicina,Grupo de Inmunología Celular e Inmunogenética,https://www.researchgate.net/profile/Mauricio_Rojas_Lopez
6,1.000,Gloria Patricia Cardona Gomez,Gloria Patricia Cardona-Gómez,University of Antioquia,Departamento de Microbiología y Parasitología,Grupo de Neurociencias de Antioquia. Cellular and Molecular Neurobiology Area,35.29,0,1966,"1,966",95,4972,Facultad de Medicina,Grupo de Neurociencias de Antioquia,https://www.researchgate.net/profile/Gloria_Cardona-Gomez
7,1.000,Sara Maria Robledo Restrepo,Sara Robledo,University of Antioquia,Instituto de Investigaciones Médicas,Instituto de Investigaciones Médicas,35.23,0,1643,"1,643",120,6100,Facultad de Medicina,Programa de Estudio y Control de Enfermedades Tropicales,https://www.researchgate.net/profile/Sara_Robledo
8,1.000,Ricardo Antonio Torres Palma,Ricardo A Torres-Palma,University of Antioquia,Instituto de Química,Instituto de Química,34.52,Professor (Full),1830,"1,830",72,10262,Facultad de Ciencias Exactas y Naturales,"Grupo de Investigación en Remediación Ambiental y Biocatálisis , Electroquímica",https://www.researchgate.net/profile/Ricardo_Torres-Palma
9,1.000,Angela Patricia Cadavid Jaramillo,Angela Cadavid,University of Antioquia,Departamento de Microbiología y Parasitología,Departamento de Microbiología y Parasitología,34.44,0,1021,"1,021",150,5754,Facultad de Medicina,"Grupo Reproducción, Grupo de Investigación en Trombosis",https://www.researchgate.net/profile/Angela_Cadavid2


In [673]:
Q.shape,QF.shape

((251, 15), (663, 15))

In [668]:
NQ.shape

(412, 15)

## Trash area
From here is for checks and code explorations

#    Implementación

1. Make the query with full name
2. Loop over the pages
3. Select affiliation Universidad de Antioquia
4. Check the name against full name
5. Get the RG profile.